In [52]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler

import numpy as np 
import matplotlib.pyplot as plt 
import argparse
import os
import cv2 
import random
import sys
from PIL import Image
import pickle
import math
import pandas as pd

In [27]:
DATADIR = "./finaldata" 
CATEGORIES = ["Boring", "Interesting"]
training_data = []
feature_extraction_data = []
IMG_SIZE = 30

In [28]:
## Stitch the images together 
## 0 1 2 
## 3 4 5
## 6 7 8

def stitch_images(file_path, file_name):
    images = [Image.open(image) for image in [file_path + "/" + file_name + str(x) + ".png" for x in range(100, 109)]]
    widths, heights = zip(*(i.size for i in images))
    total_width = int(sum(widths) / 3)
    total_height = int(sum(heights) / 3)
    new_image = Image.new("RGB", (total_width, total_height))
    for index in range(0, 9):
        image = images[index]
        new_image.paste(image, ((index % 3) * image.size[0], math.floor(index / 3) * image.size[1]))
    if not os.path.isdir(file_path + "/combined/"):
        os.mkdir(file_path + "/combined/")
    IMAGE_DIR = os.path.join(file_path, "combined/") + file_name + "combined.png"
    new_image.save(IMAGE_DIR)
    return IMAGE_DIR

In [29]:
def create_training_data():
    for category in CATEGORIES: 
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for image in sorted(os.listdir(path)): 
            if "100" not in image or "combined" in image: ## Find the starting frame
                continue
            IMAGE_DIR = stitch_images(path, image[0:-7])
            try: 
                img_array = cv2.imread(IMAGE_DIR, cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                # print(new_array.shape[0], new_array.shape[1])
                feature = np.reshape(new_array, (new_array.shape[0] * new_array.shape[1]))
                training_data.append([new_array, class_num])
                if class_num == 0:
                    feature_extraction_data.append([feature, False])
                else: 
                    feature_extraction_data.append([feature, True])
            except Exception as e: 
                pass 

In [30]:
create_training_data()
random.shuffle(training_data)
random.shuffle(feature_extraction_data)

In [31]:
features_file = open("file.csv", "ab")
feature_data = []
for i in range(len(feature_extraction_data)): 
    row_data = np.append(feature_extraction_data[i][0], feature_extraction_data[i][1])
    feature_data = np.append(feature_data, row_data)

In [44]:
feature_data.shape

(2100, 901)

In [41]:
feature_data = feature_data.reshape(2100, 901)

In [54]:
df = pd.DataFrame(feature_data)

In [56]:
df.to_csv('features.csv', index=False)  

In [49]:
np.savetxt("features.csv", feature_data)

In [50]:
feature_data[0]

array([255.,  38., 188., 218., 229., 188., 100., 202., 255., 255., 255.,
        38., 220., 243., 130., 218., 142., 214., 217.,   0., 255.,  74.,
       241., 255., 159., 234., 173., 223., 217.,   0.,  13., 252.,  68.,
       180., 221., 247., 255.,  31., 227., 210.,  40., 255., 107.,  32.,
       245., 255.,  89.,  68., 248., 238.,  89., 255., 152.,  72., 255.,
       255.,   0.,  97.,  35., 255.,  99., 255., 181., 218.,  90., 129.,
       110., 208.,  77., 122., 135., 255., 215., 243., 135., 172., 148.,
       237., 116., 165., 163., 255., 241., 255., 172., 208., 173., 255.,
       160., 202., 252., 212.,  64.,  32., 115., 140., 154., 191., 208.,
       238., 254., 240.,  31.,  72.,   0.,  19., 180., 191., 237., 255.,
       255., 217.,  79., 119.,  23.,  64.,  32.,  10., 255., 255.,  12.,
        77., 250., 243.,   0., 186., 255., 191., 252., 129.,  52., 110.,
        64., 255.,  19., 209.,   0.,   0.,  38., 172.,  99., 152.,  31.,
       255.,  64., 114.,  42.,   0.,  36., 208., 25

In [33]:
1892100/2100

901.0